In [32]:
import pandas as pd
import pickle

In [33]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [34]:
dataset=pd.get_dummies(dataset,drop_first =True)

In [35]:
dataset.columns

Index(['User ID', 'Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [36]:
indep=dataset[['User ID', 'Age', 'EstimatedSalary','Gender_Male']]

In [37]:
dep=dataset[["Purchased"]]

In [38]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(indep,dep,test_size=0.30,random_state=0)

In [39]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [41]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
parm={'solver':["lbfgs","liblinear","newton-cg","saga"]}
grid=GridSearchCV(LogisticRegression(),parm,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
grid.fit(x_train,y_train)

C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Done  10 out of  12 | elapsed:    2.8s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    2.8s finished
C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid={'solver': ['lbfgs', 'liblinear', 'newton-cg', 'saga']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1_weighted', verbose=3)

In [42]:
re=grid.cv_results_

In [43]:
y_pred=grid.predict(x_test)

In [46]:
from sklearn.metrics import confusion_matrix
cfm=confusion_matrix(y_test,y_pred)
print(cfm)

[[75  4]
 [10 31]]


In [48]:
from sklearn.metrics import classification_report
clr=classification_report(y_test,y_pred)
print(clr)

              precision    recall  f1-score   support

           0       0.88      0.95      0.91        79
           1       0.89      0.76      0.82        41

    accuracy                           0.88       120
   macro avg       0.88      0.85      0.87       120
weighted avg       0.88      0.88      0.88       120



In [52]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,y_pred,average='weighted')
print("The f1 score for the best parameter{}:".format(grid.best_params_),f1_macro)

The f1 score for the best parameter{'solver': 'lbfgs'}: 0.8808622165169021


In [54]:
from sklearn.metrics import roc_auc_score
roc=roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])
print(roc)

0.9475146650200679


In [57]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_solver,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.013336,0.004717,0.001667,0.002357,lbfgs,{'solver': 'lbfgs'},0.815589,0.752288,0.888718,0.818854,0.055646,1
1,0.005001,0.004084,0.005001,0.004083,liblinear,{'solver': 'liblinear'},0.805615,0.752288,0.888718,0.815505,0.056040,4
2,0.018338,0.008500,0.003336,0.002359,newton-cg,{'solver': 'newton-cg'},0.815589,0.752288,0.888718,0.818854,0.055646,1
3,0.006668,0.002357,0.005002,0.000001,saga,{'solver': 'saga'},0.815589,0.752288,0.888718,0.818854,0.055646,1
